In [2]:
import networkx as nx
import numpy as np
import plotly.graph_objects as go
import matplotlib as mpl
import pandas as pd
from IPython.display import clear_output
from plotly.subplots import make_subplots
from matplotlib import pyplot as plt
from qiskit import Aer
from qiskit import QuantumCircuit
from qiskit.visualization import plot_state_city
from qiskit.algorithms.optimizers import COBYLA, SLSQP, ADAM
from time import time
from copy import copy
from typing import List
from util import display_maxcut_widget, QAOA_widget, graphs
mpl.rcParams['figure.dpi'] = 300

En este Notebook vamos a explorar mas a fondo el algoritmo QAOA aplicado al  problema MaxCut. El archivo `custom.csv` contiene una lista de vertices con pesos para los conectores. A cpntinuacion, utilizamos el modulo `network` para crear una grafica basada en los valores en el archivo.csv. El widget de visualizacion nos va a permitir explorar las soluciones, los diferentes grupos seran coloreados de azul claro y oscuro y los conectores entre diferentes grupos apareceran en color rojo.

In [3]:
graph = nx.Graph()
#Añadimos algunos vertices y conectores
graph.add_nodes_from(np.arange(0,6,1))
edges = [(0,1,2.0),(0,2,3.0),(0,3,2.0),(0,4,4.0),(0,5,1.0),(1,2,4.0),(1,3,1.0),(1,4,1.0),(1,5,3.0),(2,4,2.0),(2,5,3.0),(3,4,5.0),(3,5,1.0)]
graph.add_weighted_edges_from(edges)
graphs['custom'] = graph
#Graficamos el grafo que acabamos de crear
display_maxcut_widget(graphs['custom'])

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…

Output()

Para la grafica que acabamos de definir, vamos a encontrar la solucion al problema de MaxCut de la forma '$x=$\[$x_0, x_1, ..., x_n$\]'. A continuacion crearemos una funcion que nos permita graficar los diferentes valores resultantes de la funcion de costo. La matrix `weight_matrix` corresponde al peso asignado entre los vertices [i,j].

## Solucion clasica (brute force approach)

In [4]:
def maxcut_cost_fn(graph: nx.Graph, bitstring: List[int]) -> float:
    """
    Calcula la funcion de costo para un grafo determinado con una particion
    representada por los bits 0 o 1 en una cadena de bits
    Argumentos:
        graph: El grafo para calcular los valores de costo
        bitstring: Una cadena de valores '0' or '1' que representan la asignacion a un grupo 
        o particion
    La funcion regresa:
        El costo de la particion
    """
    #Obtener la matrix de pesos de los conectores en el grafo
    weight_matrix = nx.adjacency_matrix(graph).toarray()
    size = weight_matrix.shape[0]
    value = 0.
    
    cost = 0.
    for i in range(size):
        for j in range(size):
            cost = cost + weight_matrix[i,j]*bitstring[i]*(1 - bitstring[j])
    if value < cost:
        value = cost
        xbest_brute = bitstring
    print('caso = ' + str(bitstring)+ ' costo = ' + str(cost))
    #print('\nMejor solucion = ' + str(xbest_brute) + ' costo = ' + str(value))
    return value

def plot_maxcut_histogram(graph: nx.Graph) -> None:
    """
    Grafica un histograma de barras con todos los posibles valores correspondientes
    a una posible particion para un grafo
    Argumentos:
        graph: El grafo
    """
    num_vars = graph.number_of_nodes()
    #Empezamos creando una lista de bits y sus costos asociados
    bitstrings = ['{:b}'.format(i).rjust(num_vars, '0')[::-1] for i in range(2**num_vars)]
    print(bitstrings[0][0])
    values = [maxcut_cost_fn(graph = graph, bitstring = [int(x) for x in bitstring]) for bitstring in bitstrings]
    #Ordenar la lista en orden descendente en valor de costo
    values, bitstrings = zip(*sorted(zip(values, bitstrings)))
    #Graficar el costo asociado
    bar_plot = go.Bar(x = bitstrings, y = values, marker=dict(color=values, colorscale = 'plasma', colorbar=dict(title='Costo')))
    fig = go.Figure(data=bar_plot, layout = dict(xaxis=dict(type = 'category'), width = 1500, height = 600))
    fig.show()


Despues de definir estas funciones de herramienta, calculamos el costo para todas las posibles particiones de nuesta grafica, todas las posibles asignaciones de 0 o 1 para cada vertice en el grafo y su costo asociado.

In [5]:
plot_maxcut_histogram(graph = graphs['custom'])

0
caso = [0, 0, 0, 0, 0, 0] costo = 0.0
caso = [1, 0, 0, 0, 0, 0] costo = 12.0
caso = [0, 1, 0, 0, 0, 0] costo = 11.0
caso = [1, 1, 0, 0, 0, 0] costo = 19.0
caso = [0, 0, 1, 0, 0, 0] costo = 12.0
caso = [1, 0, 1, 0, 0, 0] costo = 18.0
caso = [0, 1, 1, 0, 0, 0] costo = 15.0
caso = [1, 1, 1, 0, 0, 0] costo = 17.0
caso = [0, 0, 0, 1, 0, 0] costo = 9.0
caso = [1, 0, 0, 1, 0, 0] costo = 17.0
caso = [0, 1, 0, 1, 0, 0] costo = 18.0
caso = [1, 1, 0, 1, 0, 0] costo = 22.0
caso = [0, 0, 1, 1, 0, 0] costo = 21.0
caso = [1, 0, 1, 1, 0, 0] costo = 23.0
caso = [0, 1, 1, 1, 0, 0] costo = 22.0
caso = [1, 1, 1, 1, 0, 0] costo = 20.0
caso = [0, 0, 0, 0, 1, 0] costo = 12.0
caso = [1, 0, 0, 0, 1, 0] costo = 16.0
caso = [0, 1, 0, 0, 1, 0] costo = 21.0
caso = [1, 1, 0, 0, 1, 0] costo = 21.0
caso = [0, 0, 1, 0, 1, 0] costo = 20.0
caso = [1, 0, 1, 0, 1, 0] costo = 18.0
caso = [0, 1, 1, 0, 1, 0] costo = 21.0
caso = [1, 1, 1, 0, 1, 0] costo = 15.0
caso = [0, 0, 0, 1, 1, 0] costo = 11.0
caso = [1, 0, 0, 1, 1, 0]

## La version cuantica (el algoritmo QAOA)

El ejercicio ahora es escribir la funcion que toma como argumentos una expresion cuadratica y el parametro p y produce un circuito QAOA con $p$ capas. En la funcion de abajo solo falta insertar los circuitos que corresponden al operador `mixer` y `problema`. La funcion cuadratica, tipo QUBO es generada mediante la siguiente funcion `quadratic_program_from_graph` 

In [6]:
from qiskit.optimization.applications.ising import max_cut
from qiskit.optimization.problems import QuadraticProgram

def quadratic_program_from_graph(graph: nx.Graph) -> QuadraticProgram:
    """Construye una expresion tipo QUBO a partir de un grafo
    para obtener una solucion al problema MaxCut.
    Argumentos:
        graph: Grafo.
    La funcion regresa:
        Expresion tipo QUBO
    """
    weight_matrix = nx.adjacency_matrix(graph)
    shape = weight_matrix.shape
    size = shape[0]
    #Construir la matrix QUBO desde la matriz de pesos W
    qubo_matrix = np.zeros((size, size))
    qubo_vector = np.zeros(size)
    for i in range(size):
        for j in range(size):
            qubo_matrix[i, j] -= weight_matrix[i, j]
    for i in range(size):
        for j in range(size):
            qubo_vector[i] += weight_matrix[i,j]

    quadratic_program = QuadraticProgram()
    _ = [quadratic_program.binary_var('x{}'.format(i)) for i in range(size)]
    quadratic_program.maximize(linear=qubo_vector, quadratic=qubo_matrix)
    
    return quadratic_program

/opt/anaconda3/lib/python3.8/site-packages/qiskit/optimization/__init__.py:92: DeprecationWarning:

The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>



La siguiente funcion construye el algoritmo QAOA, inserta las partes que faltan para construir los operadores `mixer` y `problema`.

In [7]:
def qaoa_circuit(qubo: QuadraticProgram, p: int = 1):
    """
    Construye el circuito QAOA con p capas a partir de una expresion qubo.
    Argumentos:
        qubo: La expresion qubo
        p: El numero de capas en el circuito QAOA
    Regresa:
        El circuito QAOA
    """
    size = len(qubo.variables)
    qubo_matrix = qubo.objective.quadratic.to_array(symmetric=True)
    qubo_linearity = qubo.objective.linear.to_array()

    qaoa_circuit = QuantumCircuit(size,size)
    #Aplicamos la primer capa de compuertas H a todos los cubits
    qaoa_circuit.h(range(size))

    #Creamos los parametros del circuito
    gammas = ParameterVector('gamma', p)
    betas = ParameterVector('beta', p)

    #Outer loop to create each layer
    for i in range(p):
        for j in range(size):
            ro = 0
            for k in range(size):
                ro=ro+qubo_matrix[j,k]
            #Aplicar las compuertas rotacionales R_Z de la funcion de costo
            #INSERTA TU CODIGO AQUI
        for j in range(size):
            for k in range(size):
                if k!=j:
                    # Aplicar las compuertas R_ZZ rotacionales
                    # INSERTA TU CODIGO AQUI
        for j in range(size):
            # Aplicar las compuertas X con angulo 2*beta_i a todos los cubits
            # INSERTA TU CODIGO AQUI
            qaoa_circuit.rx(2*betas[i], j)

    return qaoa_circuit

IndentationError: expected an indented block (<ipython-input-7-2cdaf59758d6>, line 36)

Una vez terminado el codigo, lo puedes probar corriendo los comandos que siguen

In [ ]:
quadratic_program = quadratic_program_from_graph(graphs['custom'])
custom_circuit = qaoa_circuit(qubo = quadratic_program)

In [ ]:
test = custom_circuit.assign_parameters(parameters=[1.0]*len(custom_circuit.parameters))